# Data Wrangling: Acquisition & Preparation

In this lesson, we'll be acquiring and preparing some data from our SQL database.

## Learning Goals:

    
- Acquire a sample of data from SQL.
- Identify null values, which nulls are 'deal-breakers', i.e. rows removed, which nulls should be represented by 0, and which should be replaced by a value from other methods, such as mean.		
- Identify outliers and decide what to do with them, if anything (remove, keep as-is, replace).
- Data Structure: Aggregate as needed so that every row is an observation and each column is a variable (1 variable and not a measure). 

In [ ]:
# regular imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import env
# from acquire import get_mallcustomer_data()

In [ ]:
# from our acquire.py:
def get_connection(db, user=env.user, host=env.host, password=env.password):
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'
    
def get_mallcustomer_data():
    df = pd.read_sql('SELECT * FROM customers;', get_connection('mall_customers'))
    return df.set_index('customer_id')

In [ ]:
# acquire our dataframe

In [ ]:
df = get_mallcustomer_data()

In [ ]:
type(df)

### Acquire & Summarize

In [ ]:
# How do we get a summarization of our data?

In [ ]:
# summary of data:
# .head()
# .info()
# .describe()
# value_counts() *spread of data
# observe nulls in the dataframe
#      deal with nulls accordingly:
#      -fillna(0)
#      -fillna(value/mean/mode)//sklearn.imputer(only after split)
#      -drop

In [ ]:
df.info()

In [ ]:
def nulls_by_col(df):
    num_missing = df.isnull().sum()
    rows = df.shape[0]
    prcnt_miss = num_missing / rows * 100
    cols_missing = pd.DataFrame({'num_rows_missing': num_missing, 'percent_rows_missing': prcnt_miss})
    return cols_missing

In [ ]:
nulls_by_col(df)

In [ ]:
percent_missing

In [ ]:
df.shape[0]

In [ ]:
# pandas axes:
# rows: r0ws
# cols: co1s

In [ ]:
def nulls_by_row(df):
    num_missing = df.isnull().sum(axis=1)
    prcnt_miss = num_missing / df.shape[1] * 100
    rows_missing = pd.DataFrame({'num_cols_missing': num_missing, 'percent_cols_missing': prcnt_miss})\
    .reset_index()\
    .groupby(['num_cols_missing', 'percent_cols_missing']).count()\
    .rename(index=str, columns={'customer_id': 'num_rows'}).reset_index()
    return rows_missing

In [ ]:
print(nulls_by_row(df).to_markdown())

In [ ]:
def summarize(df):
    '''
    summarize will take in a single argument (a pandas dataframe) 
    and output to console various statistics on said dataframe, including:
    # .head()
    # .info()
    # .describe()
    # value_counts()
    # observation of nulls in the dataframe
    '''
    print('=====================================================\n\n')
    print('Dataframe head: ')
    print(df.head(3).to_markdown())
    print('=====================================================\n\n')
    print('Dataframe info: ')
    print(df.info())
    print('=====================================================\n\n')
    print('Dataframe Description: ')
    print(df.describe().to_markdown())
    num_cols = [col for col in df.columns if df[col].dtype != 'O']
    cat_cols = [col for col in df.columns if col not in num_cols]
    print('=====================================================')
    print('DataFrame value counts: ')
    for col in df.columns:
        if col in cat_cols:
            print(df[col].value_counts())
        else:
            print(df[col].value_counts(bins=10, sort=False))
    print('=====================================================')
    print('nulls in dataframe by column: ')
    print(nulls_by_col(df))
    print('=====================================================')
    print('nulls in dataframe by row: ')
    print(nulls_by_row(df))
    print('=====================================================')

In [ ]:
summarize(df)

In [ ]:
df.hist()
plt.tight_layout()
plt.show()

In [ ]:
# hypothetical // practical removal of nulls:

In [ ]:
def remove_columns(df, cols_to_remove):
    df = df.drop(columns=cols_to_remove)
    return df

In [ ]:
def handle_missing_values(df, prop_required_columns=0.5, prop_required_row=0.75):
    threshold = int(round(prop_required_columns * len(df.index), 0))
    df = df.dropna(axis=1, thresh=threshold)
    threshold = int(round(prop_required_row * len(df.columns), 0))
    df = df.dropna(axis=0, thresh=threshold)
    return df

In [ ]:
# combining everything in a cleaning function:

def data_prep(df, cols_to_remove=[], prop_required_column=0.5, prop_required_row=0.75):
    df = remove_columns(df, cols_to_remove)
    df = handle_missing_values(df, prop_required_column, prop_required_row)
    return df

In [ ]:
df = data_prep(df)

In [ ]:
df.info()

In [ ]:
# Handling Outliers:
# note your use-case! (handle outliers approrpriately. 
# ( Do we want to drop them? )
# z-score: appropriate for normal data, based on normal distribution
# Tukey: utilizing fences with inner quartile range, not contingent on normality

In [ ]:
# to utilize tukeys method:
# calculate iqr:
# get Q3 and Q1, get difference (q3 - q1)
# establish fences: 
## standard inner fence: 1.5
## standard outer fence: 3
# upper bound: q3 + k * iqr
# lower bound: q1 - k * iqr

In [ ]:
def get_upper_outliers(s, k=1.5):
    q1, q3 = s.quantile([.25, 0.75])
    iqr = q3 - q1
    upper_bound = q3 + k * iqr
    return s.apply(lambda x: max([x - upper_bound, 0]))

In [ ]:
def add_upper_outlier_columns(df, k=1.5):
    for col in df.select_dtypes('number'):
        df[col + '_outliers_upper'] = get_upper_outliers(df[col], k)
    return df

In [ ]:
df = add_upper_outlier_columns(df)

In [ ]:
df.head()

In [ ]:
outlier_cols = [col for col in df.columns if col.endswith('_outliers_upper')]
for col in outlier_cols:
    print(col, ': ')
    subset = df[col][df[col] > 0]
    print(subset.describe())

In [ ]:
# we have:
# in an organized fashion:
# acquired our data
# examined the structure and integrity of the data
# we hav observed descriptive statistics (univariate)
# we have examined null values in a nuanced fashion
# we have examined where our outliers live and assessed how we want to approach them
 # (Takeaway: our outliers exist in income, they appear to be valid and I will not 
    # drop them at this time)